## Use CNN to classify hand-drawn pictures (best model)

<img src="img/cnn.jpg" align='left'>

In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping
import numpy as np
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


PERSONAL NOTES:

Input (1 example): 2D array of pixels

Output (1 example) : array of votes (predicted label = one with highest vote)

CNN applies backpropagation using gradient descent => batch size (# of trainning examples utilised in 1 iteration)

--Factors which affect the model--

+ Convolution: # of features/filters + size of features

+ Max pooling: window size + window stride

+ Fully connected layer: # of neurons of intermidiate layers

+ Architecture: How many of each type layer + what order

+ Number of Dropout layers & Dropout values

+ etc.

Rule of thumb: if your data still useful after swapping the colums/rows then CNN is not for it

*Strides = how many pixels the pooling window move accross the filtered image

*reLU: activation function

*Batch size = the number of training examples utilised in one iteration

*Channel = 3 if RGB elif = 1 if grayscale (in our case it's the latter)

*Dropout = dropout randomly switches off some neurons in the network which forces the data to find new paths. 
    Therefore, this reduces overfitting

In [0]:
# make dict {0:"sink",1:"pear",...} 
categories=["sink","pear","moustache","nose","skateboard","penguin","peanut","skull","panda","paintbrush","nail","apple","rifle","mug","sailboat","pineapple","spoon","rabbit","shovel","rollerskates","screwdriver","scorpion","rhinoceros","pool","octagon","pillow","parrot","squiggle","mouth","empty","pencil"]
categories_dict = dict(enumerate(categories))
print(categories_dict)

{0: 'sink', 1: 'pear', 2: 'moustache', 3: 'nose', 4: 'skateboard', 5: 'penguin', 6: 'peanut', 7: 'skull', 8: 'panda', 9: 'paintbrush', 10: 'nail', 11: 'apple', 12: 'rifle', 13: 'mug', 14: 'sailboat', 15: 'pineapple', 16: 'spoon', 17: 'rabbit', 18: 'shovel', 19: 'rollerskates', 20: 'screwdriver', 21: 'scorpion', 22: 'rhinoceros', 23: 'pool', 24: 'octagon', 25: 'pillow', 26: 'parrot', 27: 'squiggle', 28: 'mouth', 29: 'empty', 30: 'pencil'}


In [10]:
#Run this code to extract xTrain.npy, xTest.npy, yTrain.npy
! unzip data.zip

Archive:  data_new_smaller.zip
replace x_test_38.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## Load data

In [4]:
# Load data
xTrain = np.load('xTrain.npy')
yTrain = np.array(np.load('yTrain.npy'))
print('Train data size: ',len(xTrain))
print('Train data shape: ', xTrain.shape)
xTest = np.load('xTest.npy')
print ('Test data size: ', len(xTest))



Train data size:  10000
Train data shape:  (10000, 1444)
Test data size:  10000


In [8]:
#Batch size
batch_size = 200
#Number of classes
numClasses = 31
#Number of epochs
epochs = 200

## Prepare our inputs
- Split data into train and validation set
- Convert yTrain to binary class matrices

In [9]:
# dimensions of images
rows, cols = 38, 38

# split data into train and validation sets
x_train, x_valid, y_train, y_valid = train_test_split(xTrain, yTrain,test_size=0.20,random_state=0)

x_train = x_train.reshape(8000,rows,cols,1)
x_valid = x_valid.reshape(2000,rows,cols,1)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_valid.shape[0], 'test samples')

# convert class vectors to binary class matrices
# 2 => [0,0,1,0,0,....]
# 0 => [1,0,0,0,0,....]
y_train = keras.utils.to_categorical(y_train, numClasses)
y_valid = keras.utils.to_categorical(y_valid, numClasses)


x_train shape: (8000, 38, 38, 1)
8000 train samples
2000 test samples


## BUILD CNN MODEL

In [0]:
#MODEL

# --- BUILD NETWORK ---
model = Sequential()
# 3 Convo Layers and 1 Pool layer (+Dropout)
model.add(Conv2D(92, (2, 2), padding='same', activation='relu', input_shape=(rows,cols,1)))
model.add(Conv2D(100, (2, 2), activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
# Add dropout randomly switches off some neurons in the network 
# Avoid overfitting

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# 3 Convo Layers and 1 Pool layer (+Dropout)
model.add(Conv2D(92, (2, 2), padding='same', activation='relu'))
model.add(Conv2D(100, (2, 2), activation='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (2, 2), padding='same', activation='relu'))
# Add dropout randomly switches off some neurons in the network 
# Avoid overfitting

model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

#Fully connected layers
#Flatten
model.add(Flatten())
model.add(Dense(512, activation='relu'))
# Add dropout randomly switches off some neurons in the network 
# Avoid overfitting
model.add(Dropout(0.5))

# Dense layers at the end which are used for class prediction
model.add(Dense(31, activation='softmax'))

In [30]:
#---COMPILE MODEL---
learning_rates =[1,0.1,0.01,0.001,0.0001] #tried all of them and found out 0.001 is the best
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=learning_rate[3]),
              metrics=['accuracy'])

#---AUGMENT DATA---
from keras.preprocessing.image import ImageDataGenerator

# Flippwing & Randomly shifting/rotating/shearing images to create more variations
datagen = ImageDataGenerator(
    rotation_range=5,
    shear_range = 2.2,
    width_shift_range=4.5,
    height_shift_range=4.7,
    horizontal_flip=True)

datagen.fit(x_train)

# Create ModelCheckpoint to save the best model only
checkpoint = ModelCheckpoint("weights.best.hdf5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#callbacks_list = [EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10),checkpoint]
callbacks_list = [checkpoint]

#---TRAIN MODEL---
# Generate augmented data on the fly
model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                    validation_data=(x_valid,y_valid),
                    callbacks=callbacks_list,
                    steps_per_epoch=len(x_train)/32, epochs=epochs)

# Evaluate on the validation test
score = model.evaluate(x_valid, y_valid, verbose=0)
#Validation loss
print('Validation loss:', score[0])
#Validation accuracy
print('Validation accuracy:', score[1])


Epoch 1/120
250/250 [==============================] - 30s 119ms/step - loss: 0.3687 - acc: 0.8854 - val_loss: 0.7090 - val_acc: 0.8480

Epoch 00001: val_acc improved from -inf to 0.84800, saving model to weights.best.hdf5
Epoch 2/120
250/250 [==============================] - 28s 113ms/step - loss: 0.3711 - acc: 0.8818 - val_loss: 0.7244 - val_acc: 0.8455

Epoch 00002: val_acc did not improve from 0.84800
Epoch 3/120
250/250 [==============================] - 28s 114ms/step - loss: 0.3679 - acc: 0.8831 - val_loss: 0.7008 - val_acc: 0.8480

Epoch 00003: val_acc did not improve from 0.84800
Epoch 4/120
250/250 [==============================] - 28s 113ms/step - loss: 0.3672 - acc: 0.8844 - val_loss: 0.7491 - val_acc: 0.8500

Epoch 00004: val_acc improved from 0.84800 to 0.85000, saving model to weights.best.hdf5
Epoch 5/120
250/250 [==============================] - 28s 114ms/step - loss: 0.3704 - acc: 0.8836 - val_loss: 0.7426 - val_acc: 0.8445

Epoch 00005: val_acc did not improve fro

After tuning our CNN, we were able to obtain a 85.5% accuracy on the train set and 84.9% on the validation set, which is far better than the standard NN and the baseline classifier SVM.

We obtained 85.34% accuracy on the real test set provided by Kaggle (we are the 4th team - RedWolves):

<img src="img/rank.jpg" align='left'>



In [22]:
#Load the best model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.load_weights("weights.best.hdf5")

model.evaluate(x_valid, y_valid)

2000/2000 [==============================] - 1s 601us/step


[0.7002374560832977, 0.8555]

In [0]:
# save model (not required)
model.save("cnn_12.model")

In [ ]:
# load model (not required)
new_model = keras.models.load_model("cnn_12.model")
new_model.evaluate(x_valid, y_valid)

In [ ]:
# predict
predictions_cnn = model.predict(xTest.reshape(10000,rows,cols,1))
result = []
for vector in predictions_cnn:
    result.append([np.argmax(vector)])

y_predictions_cnn = np.array(result)

In [29]:
# Output a few predictions from y_predictions_cnn
print(y_predictions_cnn[0][0])
#print(y_predictions_cnn[2][0])
#print(y_predictions_cnn[123][0])

15


## Output prediction to csv file

In [0]:
# Write predictions to csv file
# Write (id, dict_labels[each y in y_predictions_cnn])
import csv
with open("predictions_cnn12.csv","w") as f:
    writer = csv.writer(f)
    writer.writerow(["Id","Category"])
    id = 0
    for line in y_predictions_cnn:
        result = [id,categories_dict[line[0]]]
        writer.writerow(result)
        id += 1